In [1]:
# JuMP and HiGHS for building and solving optimization
import Pkg
Pkg.add("JuMP")
Pkg.add("HiGHS")
using(HiGHS)
using(JuMP)

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [2]:
# Coffee bean and blend vectors
types = [:Columbian, :Brazilian, :Sumatran] 
blends = [:Robust, :Light]

2-element Vector{Symbol}:
 :Robust
 :Light

In [5]:
#Zipped dictionaries
quantities = Dict(zip(types, [550, 450, 650]))
beanCost = Dict(zip(types, [1, 0.85, 1.55]))
blendCost = Dict(zip(blends, [4.25, 3.95]))



Dict{Symbol, Float64} with 2 entries:
  :Robust => 4.25
  :Light  => 3.95

In [6]:
#Model

coffeeModel = Model(HiGHS.Optimizer);

# Define Vars
@variable(coffeeModel, x[types,blends]>=0);


In [7]:
# constraints for robust blend
@constraint(coffeeModel, rsl, x[:Sumatran,:Robust] >= 0.6*sum(x[i,:Robust] for i in types))
@constraint(coffeeModel, rsu, x[:Sumatran,:Robust] <= 0.75*sum(x[i,:Robust] for i in types))
@constraint(coffeeModel, rcl, x[:Columbian,:Robust] >= 0.1*sum(x[i,:Robust] for i in types))

rcl : 0.9 x[Columbian,Robust] - 0.1 x[Brazilian,Robust] - 0.1 x[Sumatran,Robust] ≥ 0.0

In [9]:
# constraints for light blend
@constraint(coffeeModel, lbl, x[:Brazilian,:Light] >= 0.5*sum(x[i,:Light] for i in types))
@constraint(coffeeModel, lbu, x[:Brazilian,:Light] <= 0.6*sum(x[i,:Light] for i in types))
@constraint(coffeeModel, lsl, x[:Sumatran,:Light] <= 0.2*sum(x[i,:Light] for i in types))

lsl : -0.2 x[Columbian,Light] - 0.2 x[Brazilian,Light] + 0.8 x[Sumatran,Light] ≤ 0.0

In [10]:
#constraints for quantities
@constraint(coffeeModel, CQuant, sum(x[:Columbian,i] for i in blends) <= 550)
@constraint(coffeeModel, BQuant, sum(x[:Brazilian,i] for i in blends) <= 450)
@constraint(coffeeModel, SQuant, sum(x[:Sumatran,i] for i in blends) <= 650)

SQuant : x[Sumatran,Robust] + x[Sumatran,Light] ≤ 650.0

In [11]:
# Expression for objective
@expression(coffeeModel, objective, sum((-beanCost[i] + blendCost[j])*x[i,j] for i in types for j in blends))

3.25 x[Columbian,Robust] + 2.95 x[Columbian,Light] + 3.4 x[Brazilian,Robust] + 3.1 x[Brazilian,Light] + 2.7 x[Sumatran,Robust] + 2.4000000000000004 x[Sumatran,Light]

In [12]:
#lets maximize!

@objective(coffeeModel, Max, objective)

3.25 x[Columbian,Robust] + 2.95 x[Columbian,Light] + 3.4 x[Brazilian,Robust] + 3.1 x[Brazilian,Light] + 2.7 x[Sumatran,Robust] + 2.4000000000000004 x[Sumatran,Light]

In [14]:
print(coffeeModel)
optimize!(coffeeModel)
@show objective_value(coffeeModel);
@show value.(x)

Solving LP without presolve or with basis
Model   status      : Optimal
Objective value     :  4.9025000000e+03
HiGHS run time      :          0.00
objective_value(coffeeModel) = 4902.5
value.(x) = 2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [:Columbian, :Brazilian, :Sumatran]
    Dimension 2, [:Robust, :Light]
And data, a 3×2 Matrix{Float64}:
 323.33333333333337  226.66666666666663
 109.99999999999996  339.9999999999999
 650.0                 0.0


2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [:Columbian, :Brazilian, :Sumatran]
    Dimension 2, [:Robust, :Light]
And data, a 3×2 Matrix{Float64}:
 323.333  226.667
 110.0    340.0
 650.0      0.0